In [ ]:


import pandas as pd   
import numpy as np     



In [6]:
# CSV files load karna
call_logs = pd.read_csv("call_logs.csv")
agent_roster = pd.read_csv("agent_roster.csv")
disposition = pd.read_csv("disposition_summary.csv")

# Preview data
print("📄 Call Logs:")
print(call_logs.head(), "\n")

print("👥 Agent Roster:")
print(agent_roster.head(), "\n")

print("📋 Disposition Summary:")
print(disposition.head())


📄 Call Logs:
  call_id agent_id org_id installment_id     status  duration  \
0   C5333     A020     O2          L1826  completed      5.68   
1   C3045     A018     O1          L1996  no_answer     14.27   
2   C5803     A018     O1          L1849     failed     11.01   
3   C2139     A007     O1          L1046  connected      9.02   
4   C4814     A003     O1          L1887  completed      2.42   

            created_ts   call_date  
0  2025-04-28T15:40:00  2025-04-28  
1  2025-04-28T02:41:00  2025-04-28  
2  2025-04-28T19:42:00  2025-04-28  
3  2025-04-28T09:52:00  2025-04-28  
4  2025-04-28T12:58:00  2025-04-28   

👥 Agent Roster:
  agent_id users_first_name users_last_name users_office_location org_id
0     A001      AgentFirst1      AgentLast1             Bangalore     O1
1     A002      AgentFirst2      AgentLast2                 Delhi     O1
2     A003      AgentFirst3      AgentLast3                Mumbai     O1
3     A004      AgentFirst4      AgentLast4             Bangalor

In [7]:
# Step 3 - Convert 'call_date' columns into proper date format
call_logs['call_date'] = pd.to_datetime(call_logs['call_date'], errors='coerce')
disposition['call_date'] = pd.to_datetime(disposition['call_date'], errors='coerce')

# Check for missing/null values
print("📌 Missing Values:")
print("call_logs:\n", call_logs.isnull().sum(), "\n")
print("agent_roster:\n", agent_roster.isnull().sum(), "\n")
print("disposition:\n", disposition.isnull().sum())

# Check for duplicate rows
print("\n🌀 Duplicates:")
print("call_logs:", call_logs.duplicated().sum())
print("agent_roster:", agent_roster.duplicated().sum())
print("disposition:", disposition.duplicated().sum())


📌 Missing Values:
call_logs:
 call_id           0
agent_id          0
org_id            0
installment_id    0
status            0
duration          0
created_ts        0
call_date         0
dtype: int64 

agent_roster:
 agent_id                 0
users_first_name         0
users_last_name          0
users_office_location    0
org_id                   0
dtype: int64 

disposition:
 agent_id      0
org_id        0
call_date     0
login_time    3
dtype: int64

🌀 Duplicates:
call_logs: 0
agent_roster: 0
disposition: 0


In [8]:
# Step 4 - Merge call_logs with agent_roster on 'agent_id' and 'org_id'
merged1 = pd.merge(call_logs, agent_roster, on=['agent_id', 'org_id'], how='left')

# Then merge with disposition_summary on 'agent_id', 'org_id', and 'call_date'
final_df = pd.merge(merged1, disposition, on=['agent_id', 'org_id', 'call_date'], how='left')

# Preview the merged data
print("🔗 Final Merged Data Preview:")
final_df.head()


🔗 Final Merged Data Preview:


,call_id,agent_id,org_id,installment_id,status,duration,created_ts,call_date,users_first_name,users_last_name,users_office_location,login_time
0,C5333,A020,O2,L1826,completed,5.68,2025-04-28T15:40:00,2025-04-28,AgentFirst20,AgentLast20,Delhi,9:50
1,C3045,A018,O1,L1996,no_answer,14.27,2025-04-28T02:41:00,2025-04-28,AgentFirst18,AgentLast18,Bangalore,NaN
2,C5803,A018,O1,L1849,failed,11.01,2025-04-28T19:42:00,2025-04-28,AgentFirst18,AgentLast18,Bangalore,NaN
3,C2139,A007,O1,L1046,connected,9.02,2025-04-28T09:52:00,2025-04-28,AgentFirst7,AgentLast7,Bangalore,8:36
4,C4814,A003,O1,L1887,completed,2.42,2025-04-28T12:58:00,2025-04-28,AgentFirst3,AgentLast3,Mumbai,10:24


In [9]:
# Step 5 - Add presence flag
final_df['presence_flag'] = final_df['login_time'].notnull().astype(int)

# Group by agent and date, and calculate required metrics
summary = final_df.groupby(['agent_id', 'call_date']).agg(
    total_calls=('call_id', 'count'),
    connected_calls=('status', lambda x: (x == 'connected').sum()),
    avg_duration=('duration', 'mean'),
    presence_flag=('presence_flag', 'max')  # 1 if present, 0 if absent
).reset_index()

# Preview final output
summary.head()


,agent_id,call_date,total_calls,connected_calls,avg_duration,presence_flag
0,A001,2025-04-28,20,5,6.843000,1
1,A002,2025-04-28,23,4,7.827826,1
2,A003,2025-04-28,21,4,7.169524,1
3,A004,2025-04-28,27,6,8.056667,1
4,A005,2025-04-28,29,9,7.059310,0


In [10]:
# Step 6 - Export summary to CSV
summary.to_csv("dpdzero_summary_output.csv", index=False)
print("✅ Final summary saved as 'dpdzero_summary_output.csv'")


✅ Final summary saved as 'dpdzero_summary_output.csv'
